In [ ]:
from raceplotly.plots import barplot
import pandas as pd
pd.set_option("display.max_columns", None)
import tkinter as tk
from tkinter import filedialog

In [ ]:
pd.set_option("display.float_format", lambda x: "%.0f" % x)

In [ ]:
def open_file():
    root = tk.Tk()
    root.withdraw()
    path = filedialog.askopenfilename()
    df = pd.read_excel(path, None)
    sheet_list = {}
    for i, j in enumerate(list(df.keys())):
        sheet_list.setdefault(i+1, j)
        print(i+1, j)
    sheet = eval(input("Please choose the sheet you want to open:\n"))
    df = df[sheet_list[sheet]]
    return df

In [ ]:
x = open_file()

In [ ]:
df = x[(x["狀態"].str.contains("已|未")) & (x["BG"] != "Others")]

### 2020年~現在的客戶每日出貨累積變化表製作

In [ ]:
df = df[df["預交年份"] >= 2020]

In [ ]:
yearly = df.groupby(["預交日期", "Group"])[["本國幣別NTD"]].sum().reset_index()

In [ ]:
yearly["預交日期"] = yearly["預交日期"].dt.strftime("%Y-%m-%d")

In [ ]:
yearly = yearly.pivot_table(index="Group", values="本國幣別NTD", columns="預交日期").fillna(0).reset_index()

In [ ]:
yearly_name = yearly[["Group"]]

In [ ]:
yearly_data = yearly.iloc[:, 1:].cumsum(axis=1)

In [ ]:
yearly = pd.concat([yearly_name, yearly_data], axis=1)

In [ ]:
yearly.to_excel(r"C:\Users\kaihsu\Desktop\flourish.xlsx", index=False)

### 2020年~現在的客戶各BU、品名hierachical bar chart製作

In [ ]:
hier_bar = df.groupby(["Group", "預交年份", "預交月份", "BG", "品名"])[["本國幣別NTD"]].sum().reset_index()
hier_bar.to_excel(r"C:\Users\kaihsu\Desktop\hier_bar.xlsx", index=False)

### 全部歷史資料各月變化表製作

In [ ]:
test = df.groupby([pd.Grouper(key="預交日期", freq="M"), "Group", "BG"])[["本國幣別NTD"]].sum().reset_index()

In [ ]:
test["預交日期"] = test["預交日期"].dt.strftime("%Y-%m")

In [ ]:
flourish = test.pivot_table(index=["BG", "Group"], values="本國幣別NTD", columns="預交日期").fillna(0).reset_index()

In [ ]:
x = flourish.to_records()

In [ ]:
y = pd.DataFrame(x)
y.drop("index", axis=1, inplace=True)

In [ ]:
y.to_excel(r"C:\Users\kaihsu\Desktop\flourish.xlsx", index=False)

In [ ]:
my_raceplot = barplot(test,
                      item_column='Group',
                      value_column='本國幣別NTD',
                      time_column='預交日期', top_entries=3)

my_raceplot.plot(title = 'Top 10 Crops from 1961 to 2018',
                 item_label = 'Top 10 crops',
                 value_label = 'Production quantity (tonnes)',
                 frame_duration = 1000,
                initial_frame = "min")


In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/lc5415/raceplotly/main/example/FAOSTAT_data.csv')

my_raceplot = barplot(data,  item_column='Item', value_column='Value', time_column='Year')

my_raceplot.plot(item_label = 'Top 10 crops',
                 value_label = 'Production quantity (tonnes)',
                 time_label = 'Year: ', ## overwrites default `Date: `
                 frame_duration = 800)